In [1]:
using LinearAlgebra, Random, DataFrames, CSV, Plots
using StatsBase, Statistics
using JuMP, Gurobi

In [2]:
# Paths (relative to notebook structure)
restaurant_path    = "../clean_data/restaurant_data.csv"
scrap_path         = "../clean_data/food_scrap_locations.csv"
neighborhood_path  = "../clean_data/neighborhood_supply.csv"

# Read CSVs into DataFrames
restaurant_data       = CSV.read(restaurant_path, DataFrame)
food_scrap_locations  = CSV.read(scrap_path, DataFrame)
neighborhood_supply   = CSV.read(neighborhood_path, DataFrame)

# Preview the first few rows
println(first(restaurant_data, 5))
println(first(food_scrap_locations, 5))
println(first(neighborhood_supply, 5))

5×3 DataFrame
 Row │ latitude  longitude  waste   
     │ Float64   Float64    Float64 
─────┼──────────────────────────────
   1 │  40.6313   -73.9472  22593.5
   2 │  40.7144   -73.8319  54589.8
   3 │  40.7893   -73.9753  57175.8
   4 │  40.7498   -73.9728  26668.3
   5 │  40.7578   -73.9825  74368.1
5×27 DataFrame
 Row │ Borough    NTAName                  SiteName                           SiteAddr                           Hosted_By                       Open_Month  Day_Hours                          Notes                      Website                            BoroCD  CouncilDis  ct2010   BBL       BIN      Latitude  Longitude  PolicePrec  Object.ID  Location.Point                App.Android  App.iOS  X.Assembly.District  X.Congress.District  DSNY.District  X.DSNY.Section  DSNY.Zone  Senate.District 
     │ String15   String                   String                             String                             String                          String31    String                  

In [3]:
current_pantries       = CSV.read("../clean_data/Current Food Pantries.csv", DataFrame)

Row,Latitude,Longitude
,Float64,Float64
1,40.7541,-73.9933
2,40.6982,-73.9752
3,40.6964,-73.9911
4,40.7278,-74.0009
5,40.7513,-73.9953
6,40.8019,-73.9665


In [4]:
# ================================
# 3. CLEAN RESTAURANT
# ================================
# Columns: latitude | longitude | waste
rename!(restaurant_data, names(restaurant_data)[3] => :supply)

# Ensure Float64
restaurant_data.supply    = Float64.(restaurant_data.supply)
restaurant_data.latitude  = Float64.(restaurant_data.latitude)
restaurant_data.longitude = Float64.(restaurant_data.longitude)

# *** DROP RESTAURANTS WITH NEGATIVE SUPPLY ***
filter!(row -> row.supply >= 0, restaurant_data)



# ================================
# 3. CLEAN FOOD SCRAP CENTER DATA
# ================================
# rename Latitude and Longitude to latitude and longitude for consistency
rename!(food_scrap_locations, names(food_scrap_locations)[15] => :latitude)
rename!(food_scrap_locations, names(food_scrap_locations)[16] => :longitude)

# Keep only coordinates we need
food_scrap_locations.latitude = Float64.(food_scrap_locations.latitude)
food_scrap_locations.longitude = Float64.(food_scrap_locations.longitude)

# Keep only coordinate columns in food_scrap_locations
select!(food_scrap_locations, [:latitude, :longitude])

# ================================
# 4. CLEAN NEIGHBORHOOD SUPPLY DATA
# ================================
# Rename demand column for clarity
rename!(neighborhood_supply, names(neighborhood_supply)[4] => :supply_gap)

# Now neighborhood_supply.supply_gap might be String OR Float64.
# Only do replace/parse if it's strings.
if eltype(neighborhood_supply.supply_gap) <: AbstractString
    neighborhood_supply.supply_gap =
        parse.(Float64, replace.(neighborhood_supply.supply_gap, "," => ""))
end

# Demand = positive deficit, surplus -> 0
neighborhood_supply.demand = max.(0.0, -neighborhood_supply.supply_gap)

neighborhood_supply.latitude  = Float64.(neighborhood_supply.latitude)
neighborhood_supply.longitude = Float64.(neighborhood_supply.longitude)

# keep only necessary columns from neighborhood supply
select!(neighborhood_supply, [:latitude, :longitude, :demand, :Year])

# ================================
# 5. SHOW CLEANED HEADS
# ================================
println("=== Restaurants (cleaned) ===")
println(first(restaurant_data, 5))

println("\n=== Food Scrap Locations (cleaned) ===")
println(first(food_scrap_locations, 5))

println("\n=== Neighborhood Supply (cleaned) ===")
println(first(neighborhood_supply, 5))

=== Restaurants (cleaned) ===
5×3 DataFrame
 Row │ latitude  longitude  supply  
     │ Float64   Float64    Float64 
─────┼──────────────────────────────
   1 │  40.6313   -73.9472  22593.5
   2 │  40.7144   -73.8319  54589.8
   3 │  40.7893   -73.9753  57175.8
   4 │  40.7498   -73.9728  26668.3
   5 │  40.7578   -73.9825  74368.1

=== Food Scrap Locations (cleaned) ===
5×2 DataFrame
 Row │ latitude  longitude 
     │ Float64   Float64   
─────┼─────────────────────
   1 │  40.6355   -74.0228
   2 │  40.7526   -73.969
   3 │  40.7635   -74.0002
   4 │  40.762    -73.9693
   5 │  40.7174   -74.0108

=== Neighborhood Supply (cleaned) ===
5×4 DataFrame
 Row │ latitude  longitude  demand     Year  
     │ Float64   Float64    Float64    Int64 
─────┼───────────────────────────────────────
   1 │  40.8267   -73.9217  1.02143e5   2025
   2 │  40.8321   -73.8887  3.33493e5   2025
   3 │  40.8016   -73.9374  0.0         2025
   4 │  40.8469   -73.8918  1.13653e5   2025
   5 │  40.6193   -73.

In [18]:
# ================
# 6. EXTRACT VECTORS
# ================
R = nrow(restaurant_data)
D = nrow(current_pantries)
N = nrow(neighborhood_supply[neighborhood_supply.Year .== 2023, :])
Q = sum(restaurant_data.supply)

M = sum(demand)
F = 10000       # Center opening cost

S = 3 # 3 demand years


supply = restaurant_data.supply              # s_i
demand = neighborhood_supply.demand          # d_k

# ================
# 7. MANHATTAN DISTANCE FUNCTION
# ================
manhattan(lat1, lon1, lat2, lon2) = abs(lat1 - lat2) + abs(lon1 - lon2)

# ================
# 8. COST MATRICES
# ================
# cij: Restaurants (i) → Donation centers (j)
cij = [manhattan(restaurant_data.latitude[i], restaurant_data.longitude[i],
                 current_pantries.Latitude[j], current_pantries.Longitude[j])
       for i in 1:R, j in 1:D]

# cjk: Donation centers (j) → Neighborhoods (k)
cjk = [manhattan(current_pantries.Latitude[j], current_pantries.Longitude[j],
                 neighborhood_supply.latitude[k], neighborhood_supply.longitude[k])
       for j in 1:D, k in 1:N]

println("Size of cij (R x D): ", size(cij))
println("Size of cjk (D x N): ", size(cjk))

M = sum(demand)

Size of cij (R x D): (319, 6)
Size of cjk (D x N): (6, 197)


1.5243933021550444e8

In [20]:
cij

319×6 Matrix{Float64}:
 0.168886    0.0948795  0.109017   0.150154    0.168063    0.189833
 0.201143    0.159424   0.177193   0.182411    0.20032     0.22209
 0.053239    0.0912281  0.108662   0.087083    0.05791     0.0214317
 0.0248082   0.0539134  0.0716824  0.0501034   0.0239852   0.0584113
 0.0146114   0.0668522  0.0700344  0.0484554   0.0192824   0.0600593
 0.0154203   0.0585857  0.0460514  0.0244724   0.0145973   0.0840423
 0.0340908   0.053764   0.039627   0.00151004  0.0294198   0.108762
 0.00677234  0.0672337  0.0563168  0.0347378   0.00594934  0.0737769
 0.465758    0.428104   0.410335   0.431914    0.461087    0.540429
 0.0287449   0.0663989  0.0841679  0.0625889   0.0334159   0.0459258
 0.215085    0.252739   0.270508   0.248929    0.219756    0.140414
 0.0328113   0.0704653  0.0882343  0.0666553   0.0374823   0.0418594
 0.069791    0.107445   0.125214   0.103635    0.074462    0.0048797
 ⋮                                                         ⋮
 0.0259563   0.0762287  0

In [21]:


demand_2023 = neighborhood_supply[neighborhood_supply.Year .== 2023, :].demand
demand_2024 = neighborhood_supply[neighborhood_supply.Year .== 2024, :].demand
demand_2025 = neighborhood_supply[neighborhood_supply.Year .== 2025, :].demand

demand_df = DataFrame(
    y2023 = demand_2023,
    y2024 = demand_2024,
    y2025 = demand_2025
)

Row,y2023,y2024,y2025
,Float64,Float64,Float64
1,0.0,0.0,1.02143e5
2,0.0,0.0,3.33493e5
3,0.0,0.0,0.0
4,0.0,0.0,1.13653e5
5,0.0,0.0,0.0
6,82217.3,13128.3,5.06273e5
7,2.25651e5,0.0,0.0
8,9.04474e5,0.0,1.13911e5
9,0.0,0.0,0.0


In [28]:
average_demand = mean(Matrix(demand_df), dims=2)

197×1 Matrix{Float64}:
  34047.54142098
 111164.21622397
      0.0
  37884.271552896666
      0.0
 200539.58538104015
  75216.89106666666
 339461.9416397367
      0.0
      1.2123206961212999e6
      0.0
 124455.17158937
 660935.3033488033
      ⋮
      0.0
 477478.5816756173
 243880.36077310666
      0.0
      0.0
 773510.1512916334
      0.0
      0.0
 103669.25587499166
 776679.7817636967
      1.14574152677609e6
 265731.63622191997

In [31]:
function build_combined_model(w_cost::Float64, w_eq::Float64)
    model = Model(Gurobi.Optimizer)
    set_silent(model)

    # Decision variables
    @variable(model, x[1:R, 1:D] >= 0)   # restaurant -> center
    @variable(model, y[1:D, 1:N] >= 0)   # center -> neighborhood
    @variable(model, u[1:N] >= 0)        # unmet demand (for cost part)
    @variable(model, r[1:N] >= 0)        # received (for equity part)
    @variable(model, t >= 0)             # worst unmet demand

    # === Shared constraints ===

    # 1) Restaurant supply
    @constraint(model, [i in 1:R],
        sum(x[i,j] for j in 1:D) <= supply[i]
    )

    # 2) Flow conservation at centers
    @constraint(model, [j in 1:D],
        sum(y[j,k] for k in 1:N) == sum(x[i,j] for i in 1:R)
    )

    # 3) Demand balance for cost part:
    #    inflow + unmet = demand
    @constraint(model, [k in 1:N],
        sum(y[j,k] for j in 1:D) + u[k] == average_demand[k]
    )

    # 4) r_k = received = ∑_j y_jk
    @constraint(model, [k in 1:N],
        r[k] == sum(y[j,k] for j in 1:D)
    )

    # (optional, redundant but harmless) r_k ≤ d_k
    @constraint(model, [k in 1:N],
        r[k] <= demand[k]
    )

    # 5) Worst unmet demand: t ≥ d_k - r_k
    @constraint(model, [k in 1:N],
        t >= average_demand[k] - r[k]
    )

    # === Objective pieces ===

    @expression(model, cost_expr,
        sum(cij[i,j] * x[i,j] for i in 1:R, j in 1:D) +
        sum(cjk[j,k] * y[j,k] for j in 1:D, k in 1:N) +
        M * sum(u[k] for k in 1:N)
    )

    @expression(model, equity_expr, t)

    # Weighted objective: w_cost * cost + w_eq * equity
    @objective(model, Min, w_cost * cost_expr + w_eq * equity_expr)

    return model, x, y, u, r, t, cost_expr, equity_expr
end

build_combined_model (generic function with 1 method)

In [32]:
### I THINK THIS IS ALMOST THE STOCHASTIC MODEL

function build_stochastic_model(w_cost::Float64, w_eq::Float64)
    model = Model(Gurobi.Optimizer)
    set_silent(model)

    @variable(model, x[s=1:S, 1:R, 1:D] >= 0)   # restaurant -> center
    @variable(model, y[s=1:S, 1:D, 1:N] >= 0)   # center -> neighborhood
    @variable(model, u[s=1:S, 1:N] >= 0)        # unmet demand
    @variable(model, r[s=1:S, 1:N] >= 0)        # received
    @variable(model, t[s=1:S] >= 0)             # worst unmet demand
    @variable(model, z[1:D], Bin)               # center open (first stage)

    # Restaurant supply (optional: could be scenario-dependent if supply changes)
    @constraint(model, [s in 1:S, i in 1:R],
        sum(x[s,i,j] for j in 1:D) <= supply[i]
    )
    
    # Flow conservation at centers
    @constraint(model, [s in 1:S, j in 1:D],
        sum(y[s,j,k] for k in 1:N) == sum(x[s,i,j] for i in 1:R)
    )
    
    # Demand satisfaction
    @constraint(model, [s in 1:S, k in 1:N],
        sum(y[s,j,k] for j in 1:D) + u[s,k] == demand_df[k,s]
    )
    
    # r_k = received
    @constraint(model, [s in 1:S, k in 1:N],
        r[s,k] == sum(y[s,j,k] for j in 1:D)
    )
    
    # r_k ≤ demand
    @constraint(model, [s in 1:S, k in 1:N],
        r[s,k] <= demand_df[k,s]
    )
    
    # Worst unmet demand
    @constraint(model, [s in 1:S, k in 1:N],
        t[s] >= demand_df[k,s] - r[s,k]
    )
    
    # Center capacity constraint (link to first-stage z)
    @constraint(model, [s in 1:S, j in 1:D],
        sum(x[s,i,j] for i in 1:R) <= Q * z[j]
    )


    # === Objective pieces ===
    @objective(model, Min,
        sum(1/3 *
            w_cost * (
                sum(cij[i,j] * x[s,i,j] for i in 1:R, j in 1:D) +
                sum(cjk[j,k] * y[s,j,k] for j in 1:D, k in 1:N) +
                M * sum(u[k] for k in 1:N)
                ) +
            w_eq * (
                t[s]
                )
            for s in 1:S)
        )

    return model, x, y, u, r, t, z
end


build_stochastic_model (generic function with 1 method)

In [33]:
model, x, y, u, r, t, cost_expr, equity_expr = build_combined_model(1.0, 0.7)
optimize!(model)

Set parameter Username
Set parameter LicenseID to value 2697112
Academic license - for non-commercial use only - expires 2026-08-20


In [36]:
function compute_stats(model, x, y, u, r, t)

    # Solve the model
    optimize!(model)

    # === Extract solution ===
    x_val = value.(x)
    y_val = value.(y)
    u_val = value.(u)
    r_val = value.(r)
    t_val = value(t)

    # === Total quantities ===
    total_flow_rest_to_center = sum(x_val)
    total_flow_center_to_neighborhood = sum(y_val)
    total_unmet = sum(u_val)

    # === Worst-case unmet ===
    worst_unmet = t_val

    # === Distance / Cost metrics ===
    total_distance_ij = sum(cij[i,j] * x_val[i,j] for i in 1:R, j in 1:D)
    total_distance_jk = sum(cjk[j,k] * y_val[j,k] for j in 1:D, k in 1:N)
    total_distance = total_distance_ij + total_distance_jk

    # === Per-neighborhood stats ===
    received = r_val
    unmet = u_val
    demand_vs_received = [ (average_demand[k], received[k], unmet[k]) for k in 1:N ]

    # === Pack results ===
    return (
        total_flow_rest_to_center = total_flow_rest_to_center,
        total_flow_center_to_neighborhood = total_flow_center_to_neighborhood,
        total_unmet = total_unmet,
        worst_unmet = worst_unmet,
        total_distance_ij = total_distance_ij,
        total_distance_jk = total_distance_jk,
        total_distance = total_distance,
        received = received,
        unmet = unmet,
        demand_vs_received = demand_vs_received,
        objective_value = objective_value(model)
    )
end


compute_stats (generic function with 1 method)

In [37]:
stats = compute_stats(model, x, y, u, r, t)

println("=== Summary Statistics ===")
println("Total flow R→D: ", stats.total_flow_rest_to_center)
println("Total flow D→N: ", stats.total_flow_center_to_neighborhood)
println("Total distance R→D: ", stats.total_distance_ij)
println("Total distance D→N: ", stats.total_distance_jk)
println("Total distance traveled: ", stats.total_distance)
println("Total unmet demand: ", stats.total_unmet)
println("Worst-case unmet demand t: ", stats.worst_unmet)
println("Objective value: ", stats.objective_value)

=== Summary Statistics ===
Total flow R→D: 1.6034409875006378e7
Total flow D→N: 1.6034409875006378e7
Total distance R→D: 1.0656726989272174e6
Total distance D→N: 1.7118544525220827e6
Total distance traveled: 2.7775271514493003e6
Total unmet demand: 3.477870019682844e7
Worst-case unmet demand t: 2.3427649289879366e6
Objective value: 5.30164176818782e15
